# 2) Enumere possíveis problemas que poderíamos resolver utilizando machine learning neste dataset. Construa um ou mais modelos e nos mostre suas habilidades de modelar um problema: decidindo features, labels e realizando uma experimentação.

Problemas que poderiamos resolver com Machine Learning é:
*  Prever qual funcionário possui mais chances de sair da empresa;
*  Qual funcionário tem mais chances de ser promovido;

Focarei apenas no primeiro e tentarei construir um modelo que nos ajude a prever qual funcionário tem mais chances de sair da empresa, dada as variáveis disponibilizadas.

Para isso, pensei em construir um modelo de Redes Neurais Artificiais e propor sua complementação com uma análise de um modelo de Regressão Logística (Bônus). 

Redes Neurais e Regressão foram escolhidos pois, como temos uma variável classe ("left_company"), temos que usar modelos supervisionados.



## 1. Importando os pacotes

In [20]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from yellowbrick.classifier import ConfusionMatrix
from sklearn.metrics import confusion_matrix
from keras.models import Sequential
from keras.layers import Dense # Dense significa que todos os neurônios estão conectados: os de entrada com os da camada oculta,
                               # e estes com os da camada de saida;
from keras.utils import np_utils
from sklearn.metrics import accuracy_score


Carregando o banco de dados do meu Github, selecionando apenas as variáveis que julgo serem importantes para a criação do modelo:

In [2]:
url = 'https://raw.github.com/anacarolpimenta/Desafio-Rocketmat/master/HR_Engagement_Sat_Sales_UpdatedV4.0.csv'
db2 = pd.read_csv(url, sep = ",", usecols = ['Role','Percent_Remote',
                'last_evaluation','number_project','time_spend_company',
                'salary','LinkedIn_Hits','promotion_last_5years',
                'Emp_Title','Emp_Role','Emp_Position','average_montly_hours',
                "left_Company"]) 

db2.columns = db2.columns.str.lower() # Mudando todas as letras para minusculas
for columns in db2.columns:
  print(columns)

role
percent_remote
last_evaluation
number_project
average_montly_hours
time_spend_company
left_company
promotion_last_5years
salary
linkedin_hits
emp_role
emp_position
emp_title


## 2. Variáveis




### a) Recodificando as variáveis

In [3]:
# Verificando se existe algum valor Na nas nossas variáveis e o tipo delas
db2.info()
'''
RangeIndex: 14999 entries, 0 to 14998
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Role                   14999 non-null  object 
 1   Percent_Remote         14999 non-null  float64
 2   last_evaluation        14999 non-null  float64
 3   number_project         14999 non-null  int64  
 4   average_montly_hours   14999 non-null  int64  
 5   time_spend_company     14999 non-null  int64  
 6   promotion_last_5years  14999 non-null  int64  
 7   salary                 14999 non-null  object 
 8   LinkedIn_Hits          14999 non-null  int64  
 9   Emp_Role               14999 non-null  int64  
 10  Emp_Position           14999 non-null  int64  
 11  Emp_Title              14999 non-null  int64  
dtypes: float64(2), int64(8), object(2)
memory usage: 1.4+ MB
'''

'''
Para as variáveis do tipo object,iremos verificar se a ordem delas
está do jeito que utilizaremos no modelo
'''
db2.columns =db2.columns.str.lower()
sorted(db2["role"].unique().tolist())
'''
['Director',
 'Level 1',
 'Level 2-4',
 'Manager',
 'Senior Director',
 'Senior Manager',
 'VP']

'''


# Vamos reordenar nossa variável "role" do menor nível para o maior
niveis = {'Director': 5,
          'Level 1': 1,
          'Level 2-4':2,
          'Manager': 3,
          'Senior Director': 6,
          'Senior Manager': 4,
          'VP': 7}
db2['new_role'] = db2['role'].map(niveis)
sorted(db2["new_role"].unique())




# Verificando a ordem da Variável "salary"
print(sorted(db2["salary"].unique().tolist()))

'''
['high', 'low', 'medium']

'''
# Reordenando os níveis do mais baixo para o mais alto
nivel_sal = {'high': 3, 'low': 1, 'medium': 2}
db2["new_sal"] = db2["salary"].map(nivel_sal)
sorted(db2["new_sal"].unique().tolist())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14999 entries, 0 to 14998
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   role                   14999 non-null  object 
 1   percent_remote         14999 non-null  float64
 2   last_evaluation        14999 non-null  float64
 3   number_project         14999 non-null  int64  
 4   average_montly_hours   14999 non-null  int64  
 5   time_spend_company     14999 non-null  int64  
 6   left_company           14999 non-null  int64  
 7   promotion_last_5years  14999 non-null  int64  
 8   salary                 14999 non-null  object 
 9   linkedin_hits          14999 non-null  int64  
 10  emp_role               14999 non-null  int64  
 11  emp_position           14999 non-null  int64  
 12  emp_title              14999 non-null  int64  
dtypes: float64(2), int64(9), object(2)
memory usage: 1.5+ MB
['high', 'low', 'medium']


[1, 2, 3]

In [4]:
# Variável "last_evaluation"
db2["last_evaluation"].unique().tolist()

[0.36,
 0.37,
 0.38,
 0.39,
 0.4,
 0.41,
 0.42,
 0.43,
 0.44,
 0.45,
 0.46,
 0.47,
 0.48,
 0.49,
 0.5,
 0.51,
 0.52,
 0.53,
 0.54,
 0.55,
 0.56,
 0.57,
 0.58,
 0.59,
 0.6,
 0.61,
 0.62,
 0.63,
 0.64,
 0.65,
 0.66,
 0.67,
 0.68,
 0.69,
 0.7,
 0.71,
 0.72,
 0.73,
 0.74,
 0.75,
 0.76,
 0.77,
 0.78,
 0.79,
 0.8,
 0.81,
 0.82,
 0.83,
 0.84,
 0.85,
 0.86,
 0.87,
 0.88,
 0.89,
 0.9,
 0.91,
 0.92,
 0.93,
 0.94,
 0.95,
 0.96,
 0.97,
 0.98,
 0.99,
 1.0]

In [5]:
## Criando uma função que vai me permitir recodificar a variável:
def av_cat(x):
    if 0.3 < x < 0.4:
        #return "0.3 - 0.4"
        return 1
    if 0.4 <= x < 0.5:
        #return "0.4 - 0.5"
        return 2
    if 0.5 <= x < 0.6:
        #return "0.5 - 0.6"
        return 3
    if 0.6 <= x < 0.7:
        #return "0.6 - 0.7"
        return 4
    if 0.7 <= x < 0.8:
        #return "0.7 - 0.8"
        return 5
    if 0.8 <= x < 0.9:
        #return "0.8 - 0.9"
        return 6
    elif x >= 0.9:
        #return "0.9 - 1"
        return 7

In [6]:
db2["last_eval_cat"] = db2["last_evaluation"].apply(av_cat) # Aplicando a função na variável original e criando uma nova variável
db2["last_eval_cat"].unique().tolist() # Confirmando a nova variável

[1, 2, 3, 4, 5, 6, 7]

In [7]:
# Variável "percent_remote":

print(sorted(db2["percent_remote"].unique().tolist())) # Verificando os valores da variável

nivel_percent = {0.4: 40, 0.5: 50, 0.8: 80, 1:100} # Preparando os novos valores
db2["new_percent_remote"] = db2["percent_remote"].map(nivel_percent) # Alterando os valores da variável e criando uma nova variável
print(sorted(db2["new_percent_remote"].unique().tolist())) # Confirmando as alterações


[0.4, 0.5, 0.8, 1.0]
[40, 50, 80, 100]


In [8]:
db2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14999 entries, 0 to 14998
Data columns (total 17 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   role                   14999 non-null  object 
 1   percent_remote         14999 non-null  float64
 2   last_evaluation        14999 non-null  float64
 3   number_project         14999 non-null  int64  
 4   average_montly_hours   14999 non-null  int64  
 5   time_spend_company     14999 non-null  int64  
 6   left_company           14999 non-null  int64  
 7   promotion_last_5years  14999 non-null  int64  
 8   salary                 14999 non-null  object 
 9   linkedin_hits          14999 non-null  int64  
 10  emp_role               14999 non-null  int64  
 11  emp_position           14999 non-null  int64  
 12  emp_title              14999 non-null  int64  
 13  new_role               14999 non-null  int64  
 14  new_sal                14999 non-null  int64  
 15  la

### b) Criando os Previsores e a Classe

Criar uma variável com os previsores e com a classe ("left_company")

In [9]:
previsores = db2.drop(columns=["left_company", "role", "salary", "last_evaluation", "percent_remote", "linkedin_hits"])
classe = db2.left_company

In [14]:
previsores.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14999 entries, 0 to 14998
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype
---  ------                 --------------  -----
 0   number_project         14999 non-null  int64
 1   average_montly_hours   14999 non-null  int64
 2   time_spend_company     14999 non-null  int64
 3   promotion_last_5years  14999 non-null  int64
 4   emp_role               14999 non-null  int64
 5   emp_position           14999 non-null  int64
 6   emp_title              14999 non-null  int64
 7   new_role               14999 non-null  int64
 8   new_sal                14999 non-null  int64
 9   last_eval_cat          14999 non-null  int64
 10  new_percent_remote     14999 non-null  int64
dtypes: int64(11)
memory usage: 1.3 MB


Transformar a variável classe em uma dummy:

In [12]:
classe_dummy = np_utils.to_categorical(classe) # Transformar as classes em 
                                                # variáves Dummy

Fazer a divisão da base de dados em treinamento e teste:
> Aqui optei pelo método de classificação "hold-out": mais simples que a validação cruzada, mas que não me coloca sob o perigo de produzir dados super-ajustados.

In [26]:
previsores.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14999 entries, 0 to 14998
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype
---  ------                 --------------  -----
 0   number_project         14999 non-null  int64
 1   average_montly_hours   14999 non-null  int64
 2   time_spend_company     14999 non-null  int64
 3   promotion_last_5years  14999 non-null  int64
 4   emp_role               14999 non-null  int64
 5   emp_position           14999 non-null  int64
 6   emp_title              14999 non-null  int64
 7   new_role               14999 non-null  int64
 8   new_sal                14999 non-null  int64
 9   last_eval_cat          14999 non-null  int64
 10  new_percent_remote     14999 non-null  int64
dtypes: int64(11)
memory usage: 1.3 MB


In [16]:
X_treinamento, X_teste, y_treinamento, y_teste = train_test_split(previsores,
                                                                  classe_dummy,
                                                                  test_size = 0.3,
                                                                  random_state = 0)

Criar o modelo:

1.   Item da lista
2.   Item da lista


    modelo = Sequential()
Adicionar ao modelo, através da função Dense, a primeira camada oculta:
    
    modelo.add(Dense(units = 10, input_dim = 9))
        units = 10: 10 neuronios na primeira camada oculta;
        input_dim = 9: vão ter 9 variáveis/neurônios na camada de entrada;
    
      
    modelo.add(Dense(units = 2, activation = 'softmax')): camada de saída
         units = 2: pois temos 2 classes, na variável dummy criada  (classe_dummy)
         activation: função de ativação
             'softmax' : função de classificação binária (0 - 1). Vai indicar um valor de probabilidade pra cada uma das classes.

In [27]:
modelo = Sequential()
modelo.add(Dense(units = 12, input_dim = 11)) # 1a camada oculta
modelo.add(Dense(units = 9))
modelo.add(Dense(units = 7))
modelo.add(Dense(units = 5))
modelo.add(Dense(units = 2, activation = 'softmax')) # Camada de saída


In [18]:
modelo.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 10)                100       
_________________________________________________________________
dense_6 (Dense)              (None, 7)                 77        
_________________________________________________________________
dense_7 (Dense)              (None, 5)                 40        
_________________________________________________________________
dense_8 (Dense)              (None, 3)                 18        
_________________________________________________________________
dense_9 (Dense)              (None, 2)                 8         
Total params: 243
Trainable params: 243
Non-trainable params: 0
_________________________________________________________________


In [19]:
'''# Imprimir
from keras.utils.vis_utils import plot_model
'''



> modelo.compile(optimizer = 'adam', loss = 'categorical_crossentropy', 
metrics = ['accuracy'])


    optimizer: algoritmo que vai fazer o ajuste e a atualização dos pesos
        'adam' é o mais usado;
    loss: vai fazer o cálculo do erro;
        'categorical_crossentropy' é o padrão de função quando temos uma classe binária (0-1);
    metrics: como queremos visualizar os dados;
         'accuracy' é a taxa de acerto
         
modelo.fit(X_treinamento, y_treinamento, epochs = 1000,
validation_data = X_teste, y_teste)


    epochs: quantas vezes vai ser rodado o modelo;
    validation_data: banco de dados que vai ser usado para testar o modelo;

In [28]:
# Compilação da rede:

modelo.compile(optimizer = 'adam', loss = 'categorical_crossentropy', 
metrics = ['accuracy'])

In [29]:
# Treinando o modelo:
modelo.fit(X_treinamento, y_treinamento, epochs = 1000,
           validation_data = (X_teste, y_teste)) # Treinamento do modelo

Epoch 1/1000
329/329 [==============================] - 1s 2ms/step - loss: 0.9172 - accuracy: 0.7212 - val_loss: 0.5581 - val_accuracy: 0.7678
Epoch 2/1000
329/329 [==============================] - 0s 1ms/step - loss: 0.4752 - accuracy: 0.7795 - val_loss: 0.4079 - val_accuracy: 0.8067
Epoch 3/1000
329/329 [==============================] - 0s 1ms/step - loss: 0.4241 - accuracy: 0.8075 - val_loss: 0.4183 - val_accuracy: 0.8044
Epoch 4/1000
329/329 [==============================] - 0s 1ms/step - loss: 0.4108 - accuracy: 0.8156 - val_loss: 0.3678 - val_accuracy: 0.8424
Epoch 5/1000
329/329 [==============================] - 0s 1ms/step - loss: 0.4070 - accuracy: 0.8194 - val_loss: 0.3887 - val_accuracy: 0.8258
Epoch 6/1000
329/329 [==============================] - 0s 1ms/step - loss: 0.4034 - accuracy: 0.8222 - val_loss: 0.3698 - val_accuracy: 0.8436
Epoch 7/1000
329/329 [==============================] - 0s 1ms/step - loss: 0.3895 - accuracy: 0.8261 - val_loss: 0.3591 - val_accuracy:

Val_acc: accuracy no validation_data: 0.84 ou 84%

In [32]:
# Fazer as previsões:
previsoes = modelo.predict(X_teste) # Pegar cada um dos registros de X_teste e 
                                    # submeter a rede neural criada (modelo)
previsoes = (previsoes > 0.5)

In [ ]:
# Matriz de Confusão:

y_teste_matriz = [np.argmax(t) for t in y_teste] # pega o argumento máximo de
                                            # t, para o valor t em y_teste
previsao_matriz = [np.argmax(t) for t in previsoes]

confusao = confusion_matrix(y_teste_matriz, previsao_matriz)
confusao

In [46]:
taxa_acerto = accuracy_score(y_teste, previsoes) # Verificando nossa taxa de acerto
taxa_acerto

# Taxa de Acerto: 84.3%

0.8431111111111111